In [1]:
%run forward_dynamics_example.ipynb

# MIMO Nonlinear PID Position Control Simulation

In [2]:
gn = uv_body.body_restoring_vector()
ne = n - nd

buffer_sum = sum_e_buffer + ne*dt

pid = -diag(Kp)@ne - diag(Kd)@(J_@x_nb) - diag(Ki)@buffer_sum

pid_control = gn + J_.T@pid

In [3]:
pd_control_F = Function('pid_control', [W, B, r_g, r_b, Kp, Ki, Kd, sum_e_buffer, n, x_nb, nd, dt], [pid_control, buffer_sum])
pd_control_F_simpli = pd_control_F(ps.W, ps.B, ps.rg, ps.rb, Kp, Ki, Kd, sum_e_buffer, vertcat(x,y,z ,q2euler(uq)), x_nb, nd, dt)

In [4]:
pd_controller = Function('pidC', [Kp, Ki, Kd, sum_e_buffer, dt, xS0, nd], [pd_control_F_simpli[0], pd_control_F_simpli[1]])

x0 = np.array([0.0, 0.0, 0.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) #x y z w xq yq zq u v w p q r # initial conditions
xf = np.array([0.0, 0.0, 15.0, 0.0, 0.0, 0.0]) #x y z r p y # goal
kp_ = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
ki_ = np.array([0, 0, 0, 0, 0, 0])
kd_ = np.array([2,2,2,2,2,2])
sebuffer_ = np.array([0,0,0,0,0,0])
dt_ = 0.006697
pd_controller(kp_, ki_, kd_, sebuffer_ , dt_, x0, xf)

(DM([0, -0, 9.485, 0, -0, 0]), DM([0, 0, -0.100455, 0, 0, 0]))

In [5]:
# c , cpp or matlab code generation for forward dynamics
pd_controller.generate("PdController.c")
os.system(f"gcc -fPIC -shared PdController.c -o libPd.so")

0

# MIMO Nonlinear PID Velocity Control Simulation

In [6]:
ve = x_nb - vb_d

x_nb_prev = xS0_prev[7:]

dve = (x_nb_prev - x_nb)/dt

Ive = sum_e_buffer + ve*dt

v_pid = -diag(Kp)@ve - diag(Kd)@dve - diag(Ki)@Ive

v_pid_control = gn + v_pid

In [7]:
vpd_control_F = Function('v_pid_control', [W, B, r_g, r_b, Kp, Ki, Kd, sum_e_buffer, n, x_nb, xS0_prev, vb_d, dt], [v_pid_control, Ive])
vpd_control_F_simpli = vpd_control_F(ps.W, ps.B, ps.rg, ps.rb, Kp, Ki, Kd, sum_e_buffer, vertcat(x,y,z ,q2euler(uq)),x_nb, xS0_prev, vb_d, dt)

In [8]:
vpd_controller = Function('vpidC', [Kp, Ki, Kd, sum_e_buffer, dt, xS0, xS0_prev, vb_d], [vpd_control_F_simpli[0], vpd_control_F_simpli[1]])

x0 = np.array([0.0, 0.0, 0.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) #x y z w xq yq zq u v w p q r # initial conditions
xprev = np.array([0.0, 0.0, 0.0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) #x y z w xq yq zq u v w p q r # initial conditions
vf = np.array([0.0, 0.0, 0.5, 0.0, 0.0, 0.0]) #x y z r p y # goal
kp_ = np.array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])
ki_ = np.array([0, 0, 0, 0, 0, 0])
kd_ = np.array([2,2,2,2,2,2])
sebuffer_ = np.array([0,0,0,0,0,0])
dt_ = 0.006697
vpd_controller(kp_, ki_, kd_, sebuffer_ , dt_, x0, xprev, vf)

(DM([0, -0, 2.235, 0, -0, -0]), DM([0, 0, -0.0033485, 0, 0, 0]))

In [9]:
# c , cpp or matlab code generation for forward dynamics
vpd_controller.generate("vPdController.c")
os.system(f"gcc -fPIC -shared vPdController.c -o libVPd.so")

0